In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
import matplotlib.pylab as plt
from numpy import linalg as LA
%matplotlib
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using matplotlib backend: MacOSX
collins_duffy.csv
collins_duffy_retrained.csv
cosine_sim_feaure.gz
ec2-user@ec2-52-91-73-35.compute-1.amazonaws.com
sample.rtf
sample_submission.csv
semantic_pos_features.csv
stanfordData_train1.nlp
test.csv
test_with_cosine_v0.csv
train.csv
train_with_cosine_v0.csv



## Extract, Transform, Load

In [2]:
from sklearn.model_selection import train_test_split

def read_data(path_to_file):
    df = pd.read_csv(path_to_file)
    print ("Shape of base training File = ", df.shape)
    print("Shape of base training data after cleaning = ", df.shape)
    return df

df_train = read_data("input/train.csv")

df_submit = read_data("input/test.csv")
df_submit = df_submit.reset_index(drop=True)
df_submit = df_submit.reset_index()
df_submit = df_submit.rename(columns = {'index': 'mat_idx'})


# Print the column names
print (df_train.columns.values.tolist())

Shape of base training File =  (404290, 6)
Shape of base training data after cleaning =  (404290, 6)
Shape of base training File =  (2345796, 3)
Shape of base training data after cleaning =  (2345796, 3)
['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate']


## Training the Model

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc, roc_curve
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler

# Read in the pre-built features for the training set
df_treekernel = pd.read_csv('input/collins_duffy_retrained.csv')
df_semantic = pd.read_csv('input/semantic_pos_features.csv')
df_semantic.drop('is_duplicate',inplace=True,axis=1)

df_train = df_train.merge(df_treekernel,how='inner',on='id')
df_train = df_train.merge(df_semantic,how='inner',on='id')

df_train.fillna(value=0, inplace=True)
    


In [6]:
df_train.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate',
       'cdNorm_sst', 'cdNorm_st', 'S_sem_sim', 'T_sem_sim', 'sem_sim',
       'S_unmatch_n', 'T_unmatch_n', 'S_unmatch_n_p', 'T_unmatch_n_p',
       'S_unmatch_a', 'T_unmatch_a', 'S_unmatch_a_p', 'T_unmatch_a_p',
       'S_unmatch_v', 'T_unmatch_v', 'S_unmatch_v_p', 'T_unmatch_v_p',
       'S_unmatch_pp', 'T_unmatch_pp', 'S_unmatch_pp_p', 'T_unmatch_pp_p',
       'S_unmatch_wp', 'T_unmatch_wp', 'S_unmatch_wp_p', 'T_unmatch_wp_p'],
      dtype='object')

In [9]:
df_y = df_train['is_duplicate']
df_X = df_train[['cdNorm_st','cdNorm_sst','S_sem_sim', 'T_sem_sim', 'sem_sim', 'S_unmatch_n',
       'T_unmatch_n', 'S_unmatch_n_p', 'T_unmatch_n_p', 'S_unmatch_a',
       'T_unmatch_a', 'S_unmatch_a_p', 'T_unmatch_a_p', 'S_unmatch_v',
       'T_unmatch_v', 'S_unmatch_v_p', 'T_unmatch_v_p', 'S_unmatch_pp',
       'T_unmatch_pp', 'S_unmatch_pp_p', 'T_unmatch_pp_p', 'S_unmatch_wp',
       'T_unmatch_wp', 'S_unmatch_wp_p', 'T_unmatch_wp_p']]

# df_X = df_train[['sem_sim']]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.20, random_state=37)

X_train_mat = X_train.values
y_train_mat = y_train.values

X_train_mat = MinMaxScaler().fit_transform(X_train_mat)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((323425, 25), (80857, 25), (323425,), (80857,))

## Cross-Validation

In [11]:
clf = LogisticRegression()
grid = {
    'C': [1e-6, 1e-3, 1e0, 1e1],
    'penalty': ['l1', 'l2']
}
cv = GridSearchCV(clf, grid, scoring='neg_log_loss', n_jobs=-1, verbose=1)
cv.fit(X_train_mat, y_train_mat)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   26.4s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': [1e-06, 0.001, 1.0, 10.0], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=1)

In [12]:
# Print the best cross-validation parameters

print(cv.best_params_)
print(cv.best_estimator_.coef_)

{'C': 10.0, 'penalty': 'l2'}
[[  1.61754224  -2.42862706   3.42571948   3.19681045   3.88109257
   -7.54155813 -13.91593793  -0.41325342  -0.27300817  -0.26312629
    0.16219208  -1.65903536  -1.99793636   1.44386693   1.0764486
   -1.23729843  -0.98495597  -1.01711673  -1.8086452    1.60466191
    1.66443014   0.11605601   0.47894904  -0.41063815  -0.98167321]]


## ROC

In [13]:
colors = ['r', 'g', 'b', 'y', 'k', 'c', 'm', 'brown', 'r']
lw = 1
Cs = [1e-6, 1e-4, 1e0]

plt.figure(figsize=(12,8))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for different classifiers')

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')

labels = []
for idx, C in enumerate(Cs):
    clf = LogisticRegression(C = C)
    clf.fit(X_train_mat, y_train_mat)
    print("C: {}, parameters {} and intercept {}".format(C, clf.coef_, clf.intercept_))
    fpr, tpr, _ = roc_curve(y_test, clf.predict_proba(X_test)[:,1])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=lw, color=colors[idx])
    labels.append("C: {}, AUC = {}".format(C, np.round(roc_auc, 4)))

plt.legend(['random AUC = 0.5'] + labels)

C: 1e-06, parameters [[-0.00139693 -0.00350261 -0.00050778 -0.00056116 -0.00041829 -0.00468055
  -0.00218638 -0.03459075 -0.03353838 -0.00328467 -0.00247897 -0.00154138
  -0.00155417 -0.00489678 -0.00444307 -0.0022165  -0.00264604 -0.00117547
  -0.00177547 -0.00071763 -0.0011679  -0.00220256 -0.00247962 -0.00076575
  -0.00092821]] and intercept [-0.0370011]
C: 0.0001, parameters [[ 0.31510543  0.19550652  0.16886405  0.1659713   0.16837582 -0.13753654
  -0.06248332 -0.81537778 -0.75976457 -0.10098906 -0.07504494 -0.04397382
  -0.04454543 -0.1073506  -0.10318785 -0.03300168 -0.04581722 -0.02424292
  -0.04532189 -0.00689274 -0.01948956 -0.04754239 -0.0388013  -0.01184378
  -0.01116325]] and intercept [-0.16529392]
C: 1.0, parameters [[  1.6111937   -2.41933629   3.35866992   3.30566906   3.83459     -7.4467578
  -12.08231668  -0.42131599  -0.35179645  -0.32941509  -0.05626397
   -1.57985618  -1.73879561   1.35839706   0.87592908  -1.1425793
   -0.75746304  -0.89491668  -1.67986173   1.47

## Precision-Recall

In [14]:
pr, re, _ = precision_recall_curve(y_test, cv.best_estimator_.predict_proba(X_test)[:,1])
plt.figure(figsize=(12,8))
plt.plot(re, pr)
plt.title('PR Curve (AUC {})'.format(auc(re, pr)))
plt.xlabel('Recall')
plt.ylabel('Precision')

## Train Random Forest

In [101]:
# clf = RandomForestClassifier(n_estimators=50, max_features=None, max_depth=None, bootstrap=True)
# # Train the random forest on all test data. CV is "for free"
# clf.fit(X_train_mat, y_train_mat)

In [102]:
## Test on Randomly selected Subset

In [18]:
# scaler = MinMaxScaler().fit(df_sampled[['cos_sim']])
# X_samp = scaler.transform(df_sampled[['cos_sim']])

X_test_mat = MinMaxScaler().fit_transform(X_test.values)

retrained = cv.best_estimator_.fit(X_train_mat, y_train_mat)

probs = retrained.predict_proba(X_test_mat)

df_results = df_train.ix[X_test.index.values]
df_results['predictions'] = probs[:,1]




In [19]:
compare = df_results[['question1','question2','is_duplicate','predictions']]
compare

,question1,question2,is_duplicate,predictions
92172,Which colors suits on dark complexion guys? Wh...,What color of pants should I wear with a dark ...,0,0.048674
334196,Why do so many people ask questions on Quora t...,Why do some people use Quora when a dictionary...,1,0.194463
392347,How are QR codes being used for inventory and ...,How small can a QR code be?,0,0.189871
4271,How is San Diego traffic?,How often do residents of San Diego go to Mexico?,0,0.117213
112185,What are the requisites to decide the banks fi...,Who is the/a person known to have foretold wit...,0,0.002674
218879,"If the selling price is doubled, the profit tr...",What percentage of domain speculators turn a p...,0,0.017901
92360,Is time an illusion created by man?,Does time exist?,0,0.150273
327319,How can I get over a spoiled friendship?,How do I get over a broken friendship?,1,0.671047
327238,Can I register my startup company in the US wi...,Can I start a company in the US without being ...,1,0.605903
341530,"What should I do as I’m 59-years-old, divorced...",How much average profits can we expect From E ...,0,0.007423


In [16]:
from sklearn.metrics import log_loss

In [17]:
log_loss(df_results['is_duplicate'], df_results['predictions'])

0.555433336598257

In [ ]:
df_features =  df_train['']

g = sns.pairplot(df_train[['cdNorm','S_sem_sim','is_duplicate']], hue="is_duplicate", diag_kind="hist")  
for ax in g.axes.flat:  
    plt.setp(ax.get_xticklabels(), rotation=45)

In [122]:
compare

,question1,question2,is_duplicate,predictions
92172,Which colors suits on dark complexion guys? Wh...,What color of pants should I wear with a dark ...,0,0.051735
334196,Why do so many people ask questions on Quora t...,Why do some people use Quora when a dictionary...,1,0.200971
392347,How are QR codes being used for inventory and ...,How small can a QR code be?,0,0.211468
4271,How is San Diego traffic?,How often do residents of San Diego go to Mexico?,0,0.114487
112185,What are the requisites to decide the banks fi...,Who is the/a person known to have foretold wit...,0,0.004479
218879,"If the selling price is doubled, the profit tr...",What percentage of domain speculators turn a p...,0,0.018139
92360,Is time an illusion created by man?,Does time exist?,0,0.199170
327319,How can I get over a spoiled friendship?,How do I get over a broken friendship?,1,0.614926
327238,Can I register my startup company in the US wi...,Can I start a company in the US without being ...,1,0.539178
341530,"What should I do as I’m 59-years-old, divorced...",How much average profits can we expect From E ...,0,0.011884
